# Typical use case

In the minimal photoz run example notebook we demonstrated a run on the COSMOS2020 (Weaver et al. 2022) data set in order to show the most basic LePHARE functionality.

In this notebook we want to walk through a typical use case where the user wishes to run on a new catalogue with a new set of filters.

We will be looking at the same COSMOS data set but only use the ugrizy bands. We will take just these filters from the local auxiliary database. This should demonstrate the basic procedure for updating the configuration parameters and creating an input table in the appropriate format.

The notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Typical_use_case.ipynb).

In [ ]:
import lephare as lp
from astropy.table import Table
import numpy as np
import os
from matplotlib import pylab as plt
import time

%matplotlib inline
%load_ext wurlitzer

## Update the config
We will start with the COSMOS configuration as a basis. We will update the various keywords. We use the default which is shipped with lephare. You could also download the example text file config from [here](https://github.com/lephare-photoz/lephare-data/blob/main/examples/COSMOS.para) or write it completely from scratch.

In [ ]:
config = lp.default_cosmos_config.copy()

# You could also load this from a local text file:
# !curl -s -o https://raw.githubusercontent.com/lephare-photoz/lephare-data/refs/heads/main/examples/COSMOS.para
# config = lp.read_config("./COSMOS.para")

config.update(
    {
        # For a quick demonstration we use a very sparse redshift grid. DO NOT USE FOR SCIENCE!
        # Comment out the following line to improve results.
        "Z_STEP": "0.1,0.,3.",
        "VERBOSE": "NO",
    }
)

## Download the required SEDs and additional extinction laws
If one has already cloned the full auxiliary data one does not need to use this functionality.

Here we will need the same set of SEDs and other files required for the COSMOS example so will download those using the automated download functionality.

In [ ]:
lp.data_retrieval.get_auxiliary_data(
    keymap=config,
    # The additional extinction laws for galaxies are not in the principle config
    # so we must add them to be downloaded:
    additional_files=[
        "ext/SMC_prevot.dat",
        "ext/SB_calzetti.dat",
        "ext/SB_calzetti_bump1.dat",
        "ext/SB_calzetti_bump2.dat",
        # We also want the example cosmos catalogue to experiment with
        "examples/COSMOS.in",
    ],
)

## Setting new filters

As a simple example we are taking a subset of the 30 filters used in the standard COSMOS example. To do this we will take the 6 ugrizy filters from $LEPHAREDIR/filt. To use these new filters we need to update the config and set the filter directory to their location

In [ ]:
# We need to update the filter list and some other config values according to the new filter list
config.update(
    {
        # A reduced list of filters:
        "FILTER_LIST": "cosmos/u_new.pb,hsc/gHSC.pb,hsc/rHSC.pb,hsc/iHSC.pb,hsc/zHSC.pb,hsc/yHSC.pb",
        # FILTER_CALIB must be updated to either have the same length as FILTER_LIST or be one number
        # ERR_SCALE and ERR_FACTOR must also be updated later to be the correct length.
        "FILTER_CALIB": "0",
        # Use a test name to avoid clashes with other runs.
        "FILTER_FILE": "filter_test",
    }
)
filter_names = config["FILTER_LIST"].split(",")

Plot the filter transmission curves.

In [ ]:
for n, f in enumerate(filter_names):
    data = Table.read(f"{lp.LEPHAREDIR}/filt/{f}", format="ascii")
    plt.plot(data[data.colnames[0]], data[data.colnames[1]], label=f)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel("Wavelength [Angstrom]")
plt.ylabel("Transmission")

### Set specific config values for 

In order to get better results we often want to use different config values for stars, galaxies and qso.

We therefore make override dictionaries for each type. These are the default configurations that were used in Ilbert et al. 2013, Laigle et al. 2016, and Weaver et al. 2022.

In [ ]:
# We leave stars as before
star_overrides = {}

# For galaxies we want to use a different set of extinction laws and other keyword values
gal_overrides = {
    "MOD_EXTINC": "18,26,26,33,26,33,26,33",
    "EXTINC_LAW": "SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    "EM_LINES": "EMP_UV",
    "EM_DISPERSION": "0.5,1.,1.5",
}

qso_overrides = {
    "MOD_EXTINC": "0,1000",
    "EB_V": "0.,0.1,0.2,0.3",
    "EXTINC_LAW": "SB_calzetti.dat",
}

## Run prepare

These are the key preparatory stages that calculate the filters in the LePHARE format, calculate the library of SEDs and finally calculate the library of magnitudes for all the models. The prepare method runs *filter*, *sedtolib*, and *mag_gal* that would be run independently at the command line. These are all explained in detail in the [documentation](https://lephare.readthedocs.io/en/latest/original.html#detailed-lephare-user-manual).

In [ ]:
lp.prepare(
    config,
    star_config=star_overrides,
    gal_config=gal_overrides,
    qso_config=qso_overrides,
)

### Creating the input table

We need to make an astropy table as input. This can be done using the standard column order:
id, flux0, err0, flux1, err1,..., context, zspec, arbitrary_string. A simple example table with two filters might look like this:
|  id | flux_filt1  |  fluxerr_filt1 |  flux_filt2  |  fluxerr_filt2 | context | zspec | string_data |
|---|---|---|---|---|---|---|---|
|  0 | 1.e-31  | 1.e-32  | 1.e-31  | 2.e-32  | 3 | NaN | "This is just a note" |
|  1 | 2.e-31  |  1.e-32 | 1.e-31  | 2.e-32  |3 | 1. | "This has a specz" |
|  2 | 2.e-31 | 1.e-32  | 2.e-31  | 2.e-32  | 2 | NaN| "This context only uses the second filter" |

The context detemermines which bands are used but can be -99 or a numpy.nan. We do not need to have units on the flux columns but LePHARE assumes they are in erg /s /cm**2 / Hz if we are using fluxes. The number of columns must be two times the number of filters plus the four additional columns.

This input table **must use** the standard column ordering to determine column meaning. This odering depends on the filter order in the config FILTER_LIST value. 

In [ ]:
# Load the full cosmos example we downloaded at the start
cosmos_full = Table.read(f"{lp.LEPHAREDIR}/examples/COSMOS.in", format="ascii")
# Lets just look at the first 1000 specz between 0 and 3 to be fast and have a small sample to test
specz_colname = cosmos_full.colnames[-2]
mask = cosmos_full[specz_colname] > 0
mask &= cosmos_full[specz_colname] < 3
cosmos_full = cosmos_full[mask][:1000]

In [ ]:
input_table = Table()
# The id is in the first column
input_table["id"] = cosmos_full[cosmos_full.colnames[0]]
# Loop over the filters we want to keep to get the number of the filter, n, and the name, b,
for n, filter_name in enumerate(filter_names):
    # The ugrizy fluxes and errors are in cols 3 to 14
    f_col = cosmos_full.colnames[2 * n + 3]
    ferr_col = cosmos_full.colnames[2 * n + 4]
    # By default lephare uses column order so names are irrelevant
    input_table[f"f_{filter_name}"] = cosmos_full[f_col]
    input_table[f"ferr_{filter_name}"] = cosmos_full[ferr_col]
# The context is a binary flag. Here we set it to use all filters.
input_table["context"] = np.sum(2 ** np.arange(len(filter_names)))
input_table["zspec"] = cosmos_full[specz_colname]
input_table["string_data"] = "arbitrary_info"

In [ ]:
# Look at the first 5 lines of the input table
input_table[:5]

## Run process

Finally we run the main fitting process which is equivalent to *zphota* when using the command line. We also need to update some of the config values to make them consistent with the number of filters.

In [ ]:
# We will update some of the config parameters that are used during the fitting process
config.update(
    {
        # We turn on Auto adapt which uses spectroscopic redshifts to calculate zero-point
        # offsets which is crucial to getting good results.
        "AUTO_ADAPT": "YES",
        # The following measurements will correspond to all filters.
        # We could have an array of values for each.
        # If we have an array care must be taken to ensure it has a consistent length.
        "ERR_SCALE": "0.02",
        "ERR_FACTOR": "1.5",
        "SPEC_OUT": "save_spec",  # We would like to see the output
    }
)

In [ ]:
# Calculate the photometric redshifts
output, _ = lp.process(config, input_table)

In [ ]:
# the output is an astropy tabel that can be manipulated in the standard ways.
output[:5]

Next we can perform some simple plots to check the output

In [ ]:
plt.hist(output["Z_BEST"], bins=20)
plt.xlabel("redshift")

In [ ]:
plt.plot([0, 3], [0, 3], c="r")
plt.scatter(output["ZSPEC"], output["Z_BEST"], s=2.0)
plt.xlabel("z spec")
plt.ylabel("z phot")

You can use some plotting utilities provided with the code. Some examples are provided below. sel_filt is the index of the filter used to select objects by observed magnitude (starting at 0). pos_filt provides the index (starting at 0) of the filters corresponding to u, g, r, z, J, and Ks bands. If you want to create all the plots and store them in a pdf file, you can use: utils.save_photoz_plots_pdf(filename="all_photoz.pdf")

In [ ]:
utils = lp.PlotUtils(
    output,
    sel_filt=3,
    pos_filt=[0, 1, 2, 4, -1, -1],
    range_z=[0, 0.5, 1, 1.5, 3],
    range_mag=[19, 20.5, 21.5, 22.5, 25],
)

In [ ]:
utils.dist_chi2()

In [ ]:
utils.zml_zs()

In [ ]:
utils.dist_z()